In [1]:
from fastai.vision.all import *

In [2]:
pd.options.display.max_columns = 100

In [3]:
datapath = Path("/../rsna_data/")
embspath = Path(datapath/"cnn_embs")
train_df = pd.read_csv(datapath/'train.csv')

In [4]:
list(embspath.ls())

[Path('/../rsna_data/cnn_embs/full_512_FOLD1'),
 Path('/../rsna_data/cnn_embs/full_512_FOLD2'),
 Path('/../rsna_data/cnn_embs/full_512_FOLD3'),
 Path('/../rsna_data/cnn_embs/.ipynb_checkpoints'),
 Path('/../rsna_data/cnn_embs/full_512_FOLD0')]

### Load Embeddings & Preds

In [5]:
fold = 3
folddirs = [embspath/f'full_512_FOLD{fold}']; folddirs

[Path('/../rsna_data/cnn_embs/full_512_FOLD3')]

In [6]:
[o.name for o in folddirs[0].ls()]

['files.pth', 'preds.pth', 'embeddings.pth']

In [7]:
embeddings = torch.cat([torch.load(folddir/'embeddings.pth') for folddir in folddirs])
preds = torch.cat([torch.load(folddir/'preds.pth') for folddir in folddirs])
files = []
for folddir in folddirs: files += torch.load(folddir/'files.pth')

In [8]:
embeddings.shape, preds.shape, len(files)

(torch.Size([353851, 1024]), torch.Size([353851, 2]), 353851)

In [9]:
# add zero for padded input idx
input_pad_idx = len(embeddings)
embeddings = torch.cat([embeddings, torch.zeros_like(embeddings[:1])])

In [10]:
embeddings[input_pad_idx], embeddings.shape, input_pad_idx

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), torch.Size([353852, 1024]), 353851)

In [11]:
type(embeddings)

torch.Tensor

### Metadata Features

In [12]:
metadata_path = datapath/'metadata'
metadata_files = get_files(metadata_path, extensions=".csv")
metadf = pd.concat([pd.read_csv(o) for o in metadata_files]).reset_index(drop=True)

In [13]:
metadf.shape

(1790594, 68)

In [14]:
len(metadf['StudyInstanceUID'].unique())

7279

In [15]:
def minmax_scaler(o): return (o - min(o))/(max(o) - min(o))

In [16]:
scaled_pos = metadf.groupby('StudyInstanceUID')['ImagePositionPatient2'].apply(minmax_scaler)
metadf.loc[:,'scaled_position'] = scaled_pos.values

In [17]:
meta_feat_cols = ['img_min', 'img_max', 'img_mean', 'img_std', 'img_pct_window', 'scaled_position']

In [18]:
assert np.isnan(metadf[meta_feat_cols]).sum().sum() == 0

In [19]:
mean_std = metadf[meta_feat_cols].agg(['mean', 'std']).T

In [20]:
mean_std_dict = dict(zip(mean_std.index, mean_std.values.tolist())); mean_std_dict

{'img_min': [-1542.35551498553, 849.3331965009891],
 'img_max': [3209.4925326455914, 1138.112174280331],
 'img_mean': [165.7994337836255, 278.9659609535833],
 'img_std': [994.3087633304141, 293.05859626364196],
 'img_pct_window': [0.43983955119726603, 0.11747102851831802],
 'scaled_position': [0.5078721739409284, 0.29139548181397823]}

In [21]:
# standard scaler for training
for c in mean_std_dict: metadf[c] = (metadf[c] - mean_std_dict[c][0]) / mean_std_dict[c][1]

In [22]:
meta_feats_dict = dict(zip(metadf['SOPInstanceUID'], metadf[meta_feat_cols].to_numpy()))

In [23]:
len(meta_feats_dict)

1790594

In [24]:
meta_embeddings = []
for o in files:
    sopid = o.stem.split("_")[1]
    meta_embeddings.append(meta_feats_dict[sopid])
meta_embeddings = np.vstack(meta_embeddings)
meta_embeddings= tensor(meta_embeddings)

In [25]:
meta_embeddings.shape, type(meta_embeddings)

(torch.Size([353851, 6]), torch.Tensor)

In [26]:
meta_embeddings = torch.cat([meta_embeddings, torch.zeros_like(meta_embeddings[:1])])

In [27]:
embeddings.shape, meta_embeddings.shape

(torch.Size([353852, 1024]), torch.Size([353852, 6]))

In [28]:
combined_embeddings = torch.cat([embeddings, meta_embeddings], 1)

In [29]:
combined_embeddings.shape

torch.Size([353852, 1030])

In [30]:
combined_embeddings

tensor([[16.8749,  6.7855,  2.1602,  ...,  0.6463, -0.3234,  1.2081],
        [14.5586, 10.2296,  1.6654,  ...,  0.6372,  0.1519,  1.6226],
        [11.9858, 12.9292,  3.2006,  ...,  0.7038,  1.0234, -1.1626],
        ...,
        [12.7442, 18.5460,  3.4555,  ...,  0.6878,  0.3524, -1.4512],
        [14.7149,  9.9819,  1.9285,  ...,  0.3511, -1.2818,  1.6351],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

### Data

In [31]:
from fastai.text.all import *

In [32]:
image_targets = ['pe_present_on_image']
exam_targets = [
            'negative_exam_for_pe',
            'indeterminate',
            'rv_lv_ratio_gte_1',
            'rv_lv_ratio_lt_1',
            'leftsided_pe',
            'rightsided_pe',
            'central_pe',
            'chronic_pe',
            'acute_and_chronic_pe',           
             ]; exam_targets

['negative_exam_for_pe',
 'indeterminate',
 'rv_lv_ratio_gte_1',
 'rv_lv_ratio_lt_1',
 'leftsided_pe',
 'rightsided_pe',
 'central_pe',
 'chronic_pe',
 'acute_and_chronic_pe']

In [33]:
targets_df = train_df[['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID']+image_targets+exam_targets]

In [34]:
targets_dict = dict(zip(targets_df['SOPInstanceUID'].values, targets_df[image_targets+exam_targets].values))

In [35]:
len(targets_dict)

1790594

In [36]:
files_dict = defaultdict(list)
for i,o in enumerate(files):
    slice_no, sopid = o.stem.split("_")
    sid = o.parent.name
    slice_no = int(slice_no)        
    files_dict[sid].append({"slice_no":slice_no, "embs_idx":i, "img_y":targets_dict[sopid][0], "exam_y":targets_dict[sopid][1:]})

In [37]:
all_pids = list(files_dict.keys())

In [38]:
len(all_pids)

1456

In [39]:
def get_x(pid, files_dict):
    o = files_dict[pid]    
    l = sorted(o, key=lambda x: x['slice_no']) 
    return tensor([o['embs_idx'] for o in l])

def get_img_y(pid, files_dict):
    o = files_dict[pid]    
    l = sorted(o, key=lambda x: x['slice_no']) 
    img_y = [o['img_y'] for o in l]
    img_y = tensor(img_y).float()
    return img_y

def get_exam_y(pid, files_dict):
    d = files_dict[pid][0]        
    exam_y = tensor(d['exam_y']).float()
    return exam_y
    
# before_batch: after collecting samples before collating
targ_pad_idx = 666
def SequenceBlock():       return  TransformBlock(type_tfms=[partial(get_x, files_dict=files_dict)], 
                                                  dl_type=SortedDL,
                                                  dls_kwargs={'before_batch':
                                                               [partial(pad_input, pad_idx=input_pad_idx),
                                                                partial(pad_input, pad_idx=targ_pad_idx, pad_fields=1)]})
def SequenceTargetBlock(): return TransformBlock(type_tfms=[partial(get_img_y, files_dict=files_dict)])
def TargetBlock():         return TransformBlock(type_tfms=[partial(get_exam_y, files_dict=files_dict)])

In [40]:
normalized_embeddings = F.normalize(combined_embeddings, dim=0)

In [41]:
normalized_embeddings.isnan().sum()

tensor(0)

In [42]:
normalized_embeddings

tensor([[ 0.0017,  0.0010,  0.0009,  ...,  0.0011, -0.0005,  0.0020],
        [ 0.0015,  0.0015,  0.0007,  ...,  0.0011,  0.0003,  0.0027],
        [ 0.0012,  0.0019,  0.0013,  ...,  0.0012,  0.0017, -0.0020],
        ...,
        [ 0.0013,  0.0028,  0.0014,  ...,  0.0011,  0.0006, -0.0024],
        [ 0.0015,  0.0015,  0.0008,  ...,  0.0006, -0.0022,  0.0027],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

### Model

In [43]:
device = default_device(); device

device(type='cuda', index=0)

In [44]:
normalized_embeddings.shape

torch.Size([353852, 1030])

In [45]:
# emb = Embedding(*normalized_embeddings.size())
# emb.weight.requires_grad = False
# emb.weight.data.copy_(normalized_embeddings)

In [46]:
class AWD_LSTM(Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182"
    initrange=0.1

    def __init__(self, emb_sz,n_hid, n_layers, hidden_p=0.2, input_p=0.6, weight_p=0.5, bidir=False):
        store_attr('emb_sz,n_hid,n_layers')
        self.bs = 1
        self.n_dir = 2 if bidir else 1
#         self.emb_layer = emb
        
        self.rnns = nn.ModuleList([self._one_rnn(emb_sz if l == 0 else n_hid, (n_hid)//self.n_dir, bidir, weight_p, l) for l in range(n_layers)])

        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])
        self.reset()

    def forward(self, x):
#         inp = self.emb_layer(x)
        inp = normalized_embeddings[x].to(device)
        bs,sl = inp.shape[:2]
        if bs!=self.bs: self._change_hidden(bs)

        output = self.input_dp(inp)
        new_hidden = []
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            output, new_h = rnn(output, self.hidden[l])
            new_hidden.append(new_h)
            if l != self.n_layers - 1: output = hid_dp(output)
        self.hidden = to_detach(new_hidden, cpu=False, gather=False)
        return output

    def _change_hidden(self, bs):
        self.hidden = [self._change_one_hidden(l, bs) for l in range(self.n_layers)]
        self.bs = bs

    def _one_rnn(self, n_in, n_out, bidir, weight_p, l):
        "Return one of the inner rnn"
        rnn = nn.LSTM(n_in, n_out, 1, batch_first=True, bidirectional=bidir, bias=False)
        return WeightDropout(rnn, weight_p)

    def _one_hidden(self, l):
        "Return one hidden state"
        nh = (self.n_hid) // self.n_dir
        return (one_param(self).new_zeros(self.n_dir, self.bs, nh), one_param(self).new_zeros(self.n_dir, self.bs, nh))

    def _change_one_hidden(self, l, bs):
        if self.bs < bs:
            nh = (self.n_hid) // self.n_dir
            return tuple(torch.cat([h, h.new_zeros(self.n_dir, bs-self.bs, nh)], dim=1) for h in self.hidden[l])
        if self.bs > bs: return (self.hidden[l][0][:,:bs].contiguous(), self.hidden[l][1][:,:bs].contiguous())
        return self.hidden[l]

    def reset(self):
        "Reset the hidden states"
        [r.reset() for r in self.rnns if hasattr(r, 'reset')]
        self.hidden = [self._one_hidden(l) for l in range(self.n_layers)]

In [55]:
layers = [512 * 3] + [512] + [9]

class MultiHeadedSequenceClassifier(Module):
    "dim: input sequence feature dim"
    def __init__(self, bptt=72, input_pad_idx=input_pad_idx, n_meta=6, dim=1024):
        
        store_attr('input_pad_idx')
        self.encoder = SentenceEncoder(bptt=bptt, module=AWD_LSTM(1030, 512, 2, bidir=True), pad_idx=input_pad_idx)
        
        # image level preds
        self.seq_head = LinearDecoder(1, 512, bias=True)
 
        # exam level preds
        self.exam_head = PoolingLinearClassifier(layers, ps=[0.4, 0.1], bptt=bptt)
        
    
    def forward(self, x):
        import pdb; pdb.set_trace()
        out, mask = self.encoder(x) 
       
        # img level out
        seq_cls_out,_,_ = self.seq_head(out)
        seq_cls_out = seq_cls_out.squeeze(-1)
              
        # exam level out
        exam_out,_,_ = self.exam_head((out,mask))

        return (seq_cls_out, exam_out)

In [56]:
class MultiLoss(Module):
    
    def __init__(self, targ_pad_idx=666):
        store_attr("targ_pad_idx")

    def forward(self, inp, yb0, yb1):
        exam_target_weights = tensor([0.0736196319, 0.09202453988,  0.2346625767, 
                                      0.0782208589, 0.06257668712, 0.06257668712,
                                      0.1877300613,  0.1042944785, 0.1042944785]).to(yb1.device)
        seq_cls_out, exam_out = inp
       
        # img loss
        img_loss, qs = 0, 0
        mask = yb0 != self.targ_pad_idx 
        
        for _m,_y,_p in zip(mask, yb0, seq_cls_out):
            qi = _y[_m].mean()
            qs += qi*sum(_m)
            img_loss += qi*(F.binary_cross_entropy_with_logits(_p[_m], _y[_m], reduction='sum'))
        
        
        # exam loss
        exam_losses = F.binary_cross_entropy_with_logits(exam_out, yb1,reduction='none')
        tot_exam_loss = (exam_losses*(exam_target_weights.unsqueeze(0))).sum()
        tot_exam_wgts = len(exam_losses)*(tensor(exam_target_weights).sum())
        
        return (tot_exam_loss+img_loss)/(qs+tot_exam_wgts)

### Train

In [57]:
data = DataBlock(blocks=(SequenceBlock,SequenceTargetBlock,TargetBlock), 
                 n_inp=1, splitter=RandomSplitter(0.2))
dls = data.dataloaders(all_pids, bs=64)
model = SequentialRNN(MultiHeadedSequenceClassifier(bptt=256))
loss_func = MultiLoss()
learner = Learner(dls, model, loss_func=loss_func, metrics=[], cbs=[ModelResetter(), 
                                                                    TerminateOnNaNCallback(),
                                                                    SaveModelCallback(fname=f"best_sequence_model_fold{fold}")])

In [ ]:
learner.lr_find()

In [51]:
learner.fit_flat_cos(20, lr=0.005)

epoch,train_loss,valid_loss,time
0,0.648726,0.478321,00:12
1,0.467119,0.400515,00:11
2,0.372455,0.221196,00:11
3,0.323490,0.217006,00:11
4,0.291371,0.213842,00:11
5,0.268086,0.210100,00:12
6,0.251778,0.208204,00:11
7,0.239048,0.203401,00:12
8,0.231819,0.204077,00:11
9,0.229221,0.231217,00:11


Better model found at epoch 0 with valid_loss value: 0.47832125425338745.
Better model found at epoch 1 with valid_loss value: 0.4005146324634552.
Better model found at epoch 2 with valid_loss value: 0.22119557857513428.
Better model found at epoch 3 with valid_loss value: 0.21700596809387207.
Better model found at epoch 4 with valid_loss value: 0.21384158730506897.
Better model found at epoch 5 with valid_loss value: 0.21009965240955353.
Better model found at epoch 6 with valid_loss value: 0.20820404589176178.
Better model found at epoch 7 with valid_loss value: 0.20340076088905334.
Better model found at epoch 17 with valid_loss value: 0.2026049941778183.
Better model found at epoch 18 with valid_loss value: 0.20201757550239563.
Better model found at epoch 19 with valid_loss value: 0.20145009458065033.


In [52]:
learner.export(f"best_sequence_export_fold{fold}")

In [ ]:
learner.get_preds??